In [12]:
import os
import json
import numpy as np
# from nuscenes.nuscenes import NuScenes
from nuscenes.nuscenes_mbtest import NuScenes
from nuscenes.utils.splits_mbtest import create_splits_scenes
from pyquaternion import Quaternion
from pyproj import Transformer

def get_scenes(version, is_train=False):
        # filter by scene split, 
        #mb_test
        split = {
            'v1.0-trainval': {True: 'train', False: 'val'},
            'v1.0-mini': {True: 'mini_train', False: 'mini_val'},
            'mb_test': {True: 'mb_test_train', False: 'mb_test_val'},
        }[version][is_train]

        return create_splits_scenes()[split]

def get_ego_pose_from_sample_token(nusc, sample_token):
    sample = nusc.get('sample', sample_token)
    sample_data_token = sample['data']['CAM_FRONT'] # 'LIDAR_TOP'
    sample_data = nusc.get('sample_data', sample_data_token)
    ego_pose_token = sample_data['ego_pose_token']
    ego_pose = nusc.get('ego_pose', ego_pose_token)
    return ego_pose

def transform_to_global_coordinates(nusc, scene_samples_dict):
    for scene_token, samples in scene_samples_dict.items():
        for sample_dict in samples:
            for sample_token, types_dict in sample_dict.items():
                ego_pose = get_ego_pose_from_sample_token(nusc, sample_token)
                translation = np.array(ego_pose['translation'])
                rotation = Quaternion(ego_pose['rotation'])
                rotation_matrix = rotation.rotation_matrix[:2, :2]

                for type_key, points_list in types_dict.items():
                    for points in points_list:
                        for point in points:
                            local_coords = np.array([point['x'], point['y']])
                            rotated_coords = rotation_matrix.dot(local_coords)
                            global_coords = rotated_coords + translation[:2]
                            point['x'], point['y'] = global_coords[0], global_coords[1]

    return scene_samples_dict

def get_location_from_scene_token(nusc, scene_token):
    scene = nusc.get('scene', scene_token)
    log_token = scene['log_token']
    log = nusc.get('log', log_token)
    return log['location']

def convert_to_lat_lon(scene_token, maps_lat_lon, maps_width_height, point_x, point_y):
    location = get_location_from_scene_token(nusc, scene_token)
    origin_lat, origin_lon = maps_lat_lon[location]
    map_width, map_height = maps_width_height[location]
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:4326", always_xy=True) # EPSG:3857 for NuScenes
    mercator_x = origin_lon + (point_x / map_width) * 360
    mercator_y = origin_lat + (point_y / map_height) * 180
    lon, lat = transformer.transform(mercator_x, mercator_y)
    return lat, lon

def scene_samples_to_geojson(scene_samples_dict, maps_lat_lon, maps_width_height):
    geojson = {
        "type": "FeatureCollection",
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
            }
        },
        "features": []
    }

    for scene_token, samples in scene_samples_dict.items():
        for sample_dict in samples:
            for sample_token, types_dict in sample_dict.items():
                ego_pose = get_ego_pose_from_sample_token(nusc, sample_token)

                for type_key, points_list in types_dict.items():
                    if type_key == '0':
                        geometry_type = "MultiPolygon"
                        coordinates = [[[
                            convert_to_lat_lon(scene_token, maps_lat_lon, maps_width_height, point['x'], point['y'])
                            for point in points
                        ]] for points in points_list]
                    else:
                        geometry_type = "MultiLineString"
                        coordinates = [[
                            convert_to_lat_lon(scene_token, maps_lat_lon, maps_width_height, point['x'], point['y'])
                            for point in points
                        ] for points in points_list]

                    feature = {
                        "type": "Feature",
                        "geometry": {
                            "type": geometry_type,
                            "coordinates": coordinates
                        },
                        "properties": {
                            "scene_token": scene_token,
                            "sample_token": sample_token,
                            "ego_pose_token": ego_pose['token'],
                            "timestamp": ego_pose['timestamp']
                        }
                    }

                    geojson['features'].append(feature)

    return geojson

def write_scene_geojsons(scene_samples_dict, maps_lat_lon, maps_width_height, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for scene_token, samples in scene_samples_dict.items():
        geojson_data = scene_samples_to_geojson({scene_token: samples}, maps_lat_lon, maps_width_height)
        output_file = os.path.join(output_folder, f"{scene_token}.geojson")
        
        with open(output_file, 'w') as f:
            json.dump(geojson_data, f, indent=4)
        print(f"GeoJSON data for scene {scene_token} written to {output_file}")

if __name__ == '__main__':
    SUBMISSION = 'output_n005.json'
    OUTPUT_FOLDER = 'output_n005_geojsons'
    # DATAROOT = '/home/sherlock/Documents/Projects/dataset/nuScenes'
    DATAROOT = '/home/sherlock/Documents/Projects/HDMapNet-test/dataset/MB_dataset2/'
    MAPS = ['boston-seaport', 'singapore-hollandvillage', 'singapore-onenorth', 'singapore-queenstown', 'beijing-highway']
    VERSION = 'mb_test' # 'v1.0-mini' mb_test
    maps_lat_lon = {
        'boston-seaport': [42.336849169438615, -71.05785369873047],
        'singapore-onenorth': [1.2882100868743724, 103.78475189208984],
        'singapore-hollandvillage': [1.2993652317780957, 103.78217697143555],
        'singapore-queenstown': [1.2782562240223188, 103.76741409301758],
        'beijing-highway': [39.7346583, 116.4518842]
    }
    maps_width_height = {
        'singapore-onenorth': [1585.6, 2025.0],
        'singapore-hollandvillage': [2808.3, 2922.9],
        'singapore-queenstown': [3228.6, 3687.1],
        'boston-seaport': [2979.5, 2118.1],
        'beijing-highway': [709.5, 1094.0]
    }

    nusc = NuScenes(version=VERSION, dataroot=DATAROOT, verbose=False)
    # print(nusc.table_names)
    base_folder_name = os.path.splitext(SUBMISSION)[0]

    with open(SUBMISSION, 'r') as f:
        data = json.load(f)
    
    results = data.get('results', {})
    sample_tokens = list(results.keys())

    sample_data_dict = {}
    for sample_token in sample_tokens:
        sample = nusc.get('sample', sample_token)
        scene_token = sample['scene_token']
        
        if scene_token not in sample_data_dict:
            sample_data_dict[scene_token] = {}
        
        if sample_token not in sample_data_dict[scene_token]:
            sample_data_dict[scene_token][sample_token] = {
                "0": [],
                "1": [],
                "2": []
            }
        
        sample_results = results[sample_token]
        
        for result in sample_results:
            pts = result["pts"]
            type_val = str(result["type"])
            
            points_list = [{'x': pt[0], 'y': pt[1]} for pt in pts]
            
            if type_val in sample_data_dict[scene_token][sample_token]:
                sample_data_dict[scene_token][sample_token][type_val].append(points_list)
    
    scene_samples_dict = {}
    for scene_token, samples in sample_data_dict.items():
        scene_samples_dict[scene_token] = []
        for sample_token, types_dict in samples.items():
            scene_samples_dict[scene_token].append({sample_token: types_dict})
    
    scene_samples_dict = transform_to_global_coordinates(nusc, scene_samples_dict)
    write_scene_geojsons(scene_samples_dict, maps_lat_lon, maps_width_height, OUTPUT_FOLDER)


GeoJSON data for scene Sdics5PKfxaRKsQdNVAA1BWCLUT4Tmz1 written to output_n005_geojsons/Sdics5PKfxaRKsQdNVAA1BWCLUT4Tmz1.geojson


### Followed blocks just for test, i. e. not necessary

In [3]:
for scene_token, samples in sample_data_dict.items(): # 打印每个sample_token的ego_pose信息
    for sample_token in samples.keys():
        ego_pose = get_ego_pose_from_sample_token(nusc, sample_token)
        print(f"Sample Token: {sample_token}")
        print(f"Ego Pose: {ego_pose}")

Sample Token: 3e8750f331d7499e9b5123e9eb70f2e2
Ego Pose: {'token': 'fdddd75ee1d94f14a09991988dab8b3e', 'timestamp': 1533151603547590, 'rotation': [-0.968669701688471, -0.004043399262151301, -0.007666594265959211, 0.24820129589817977], 'translation': [600.1202137947669, 1647.490776275174, 0.0]}
Sample Token: 3950bd41f74548429c0f7700ff3d8269
Ego Pose: {'token': '53121f39e1a0434d8431777a05de3eec', 'timestamp': 1533151604048025, 'rotation': [-0.966388353173516, -0.003500749389185484, -0.008033331134039243, 0.2569372709343358], 'translation': [603.8259202783211, 1645.3869905049673, 0.0]}
Sample Token: c5f58c19249d4137ae063b0e9ecd8b8e
Ego Pose: {'token': '872aeb79742f4cd5b6e069c36b007ace', 'timestamp': 1533151604547893, 'rotation': [-0.9627518337207462, -0.007396822537559821, -0.006156540889652658, 0.27021526731101353], 'translation': [607.4541261458803, 1643.21492223553, 0.0]}
Sample Token: 700c1a25559b4433be532de3475e58a9
Ego Pose: {'token': 'd2e315d29ed4468f802bf6ccfdbe6852', 'timestamp':

In [12]:
# 验证转换结果
for scene_token, samples in scene_samples_dict.items():
    for sample_dict in samples:
        for sample_token, types_dict in sample_dict.items():
            print(f"Sample Token: {sample_token}")
            for type_key, points in types_dict.items():
                for point in points:
                    print(f"Global X: {point['x']}, Global Y: {point['y']}")

Sample Token: 3e8750f331d7499e9b5123e9eb70f2e2
Global X: 574.5303454517489, Global Y: 1663.2095128455203
Global X: 575.4070364748443, Global Y: 1662.72875448963
Global X: 576.2837274979397, Global Y: 1662.24799613374
Global X: 577.1604185210352, Global Y: 1661.7672377778497
Global X: 578.0371095441307, Global Y: 1661.2864794219595
Global X: 578.9138005672262, Global Y: 1660.8057210660693
Global X: 579.7904915903216, Global Y: 1660.324962710179
Global X: 580.6671826134171, Global Y: 1659.8442043542889
Global X: 581.5438736365126, Global Y: 1659.3634459983987
Global X: 582.420564659608, Global Y: 1658.8826876425085
Global X: 583.2972556827035, Global Y: 1658.4019292866183
Global X: 584.1739467057989, Global Y: 1657.921170930728
Global X: 585.0506377288945, Global Y: 1657.4404125748379
Global X: 585.9273287519899, Global Y: 1656.9596542189477
Global X: 586.8040197750854, Global Y: 1656.4788958630575
Global X: 587.6807107981808, Global Y: 1655.9981375071673
Global X: 588.5574018212764, Glo

In [13]:
# 打印结果
for scene_token, tokens_list in scene_samples_dict.items():
    print(f"Scene Token: {scene_token}")
    print("Sample Tokens and their structures:")
    for sample_structure in tokens_list:
        print(sample_structure)

Scene Token: fcbccedd61424f1b85dcbf8f897f9754
Sample Tokens and their structures:
{'3e8750f331d7499e9b5123e9eb70f2e2': {'0': [{'x': 574.5303454517489, 'y': 1663.2095128455203}, {'x': 575.4070364748443, 'y': 1662.72875448963}, {'x': 576.2837274979397, 'y': 1662.24799613374}, {'x': 577.1604185210352, 'y': 1661.7672377778497}, {'x': 578.0371095441307, 'y': 1661.2864794219595}, {'x': 578.9138005672262, 'y': 1660.8057210660693}, {'x': 579.7904915903216, 'y': 1660.324962710179}, {'x': 580.6671826134171, 'y': 1659.8442043542889}, {'x': 581.5438736365126, 'y': 1659.3634459983987}, {'x': 582.420564659608, 'y': 1658.8826876425085}, {'x': 583.2972556827035, 'y': 1658.4019292866183}, {'x': 584.1739467057989, 'y': 1657.921170930728}, {'x': 585.0506377288945, 'y': 1657.4404125748379}, {'x': 585.9273287519899, 'y': 1656.9596542189477}, {'x': 586.8040197750854, 'y': 1656.4788958630575}, {'x': 587.6807107981808, 'y': 1655.9981375071673}, {'x': 588.5574018212764, 'y': 1655.517379151277}, {'x': 589.43409

In [20]:
scene_samples_dict

{'fcbccedd61424f1b85dcbf8f897f9754': [{'3e8750f331d7499e9b5123e9eb70f2e2': {'0': [{'x': 574.5303454517489,
      'y': 1663.2095128455203},
     {'x': 575.4070364748443, 'y': 1662.72875448963},
     {'x': 576.2837274979397, 'y': 1662.24799613374},
     {'x': 577.1604185210352, 'y': 1661.7672377778497},
     {'x': 578.0371095441307, 'y': 1661.2864794219595},
     {'x': 578.9138005672262, 'y': 1660.8057210660693},
     {'x': 579.7904915903216, 'y': 1660.324962710179},
     {'x': 580.6671826134171, 'y': 1659.8442043542889},
     {'x': 581.5438736365126, 'y': 1659.3634459983987},
     {'x': 582.420564659608, 'y': 1658.8826876425085},
     {'x': 583.2972556827035, 'y': 1658.4019292866183},
     {'x': 584.1739467057989, 'y': 1657.921170930728},
     {'x': 585.0506377288945, 'y': 1657.4404125748379},
     {'x': 585.9273287519899, 'y': 1656.9596542189477},
     {'x': 586.8040197750854, 'y': 1656.4788958630575},
     {'x': 587.6807107981808, 'y': 1655.9981375071673},
     {'x': 588.5574018212764